In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import ultralytics
import kornia.geometry

In [ ]:
import electricmayhem.whitebox as em
from electricmayhem.blackbox import BlackBoxPatchTrainer

In [ ]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat',
                'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench','bird', 'cat',
                'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack',
                'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
                'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book',
                'clock', 'vase', 'scissors', 'teddy bear', 'hair drier','toothbrush']

In [ ]:
x = em.load_to_tensor("data/toycar/medium_distance_arc/057.png")
em.plot(x)

In [ ]:
import electricmayhem.blackbox
electricmayhem.blackbox._augment.augment_image(x).shape

In [ ]:
model = ultralytics.YOLO("yolov8n.pt").model.eval()

In [ ]:
def detect_func(img, return_raw=False, **kwargs):
    img = kornia.geometry.transform.resize(img, (640,640))
    model_outputs = model(img.unsqueeze(0))[0]
    class_probs = model_outputs[:,4:,:]
    maxval = torch.max(class_probs).item()
    if return_raw:
        return [int(maxval>0.25),maxval]
    else:
        return maxval

In [ ]:
detect_func(x)

In [ ]:
mask = torch.zeros_like(x)
mask[:,300:600,50:450] += 1
em.plot(x*(1-mask) + mask)

In [ ]:
SAVE_EVERY = 190
def eval_func(writer, counter, img, mask, perturbation, **kwargs):
    #print(counter % SAVE_EVERY)
    if counter % SAVE_EVERY == 0:
        pert = kornia.geometry.transform.resize(perturbation, (640,640))
        x = img*(1-mask) + mask*pert
        detections = model(x.unsqueeze(0))
        detections_converted = em._yolo.convert_ultralytics_to_v5_format(detections[0])[0]
        fig = em._yolo.plot_detections(x.unsqueeze(0), detections_converted, 0, classnames=COCO_CLASSES)
        writer.add_figure("detections", fig, global_step=counter)


In [ ]:
d = 120 #80 # 40 #20
perturbation = torch.tensor(np.random.uniform(0, 1, size=(3,d,d)).astype(np.float32))
foo = perturbation.clone().detach().numpy()
trainer = BlackBoxPatchTrainer(
    x,
    mask,
    mask,
    detect_func,
    "/media/joe/data/electricmayhem_tutorial_logs/06_d_120_q_10_beta_01/",
    perturbation=perturbation,
    num_augments=1, # number of augmentations to test each random direction under
    q=10, # number of random directions per step
    beta=0.1, #1.0, # RGF smoothing parameter
    reduce_mask=False,
    #tr_thresh=1, # transform robustness threshold for mask reduction. set high to skip that step for this example
    eval_augments=50, # number of random augmentations to use for evaluation
    use_scores=True, # use soft outputs of the model rather than hard decisions
    eval_func=eval_func,
    extra_params={"perturbation_size":d},
    aug_params={"scale":(0.99,1.01), "blur":[0], "rotate":0, "angle":0, "translate":0, "gamma":(1,1.1),
               "perspective_scale":0},
    subset_frac=0., # 0
)

In [ ]:
#trainer.fit(budget=10000, lrs=[1e5, 1e4, 1000., 100., 10., 1.])
trainer.fit(budget=1000000, lrs=[1e5, 1e4, 1000., 100., 10., 1.])